In [1]:
import librosa
import csv
import numpy as np
import pandas as pd

In [2]:
train=pd.read_csv('train.csv',on_bad_lines='skip')
train=train.dropna()
check=pd.read_csv("check.csv",on_bad_lines='skip')
check=check.dropna()

In [3]:
train.head()

,label,data1,data2,data3,data4,data5,data6,data7,data8,data9,...,data47095,data47096,data47097,data47098,data47099,data47100,data47101,data47102,data47103,data47104
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000610,0.000854,0.000549,0.001007,0.000854,0.000610,0.000885,0.000519,0.001251,0.000427
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.000092,0.000092,-0.000122,-0.000336,-0.000122,-0.000397,-0.000061,-0.000549,0.000580,-0.000305
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001404,0.001495,0.001312,0.000610,0.001495,0.001343,0.000824,0.001160,0.000488,0.001160
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.002930,0.001923,0.002472,0.002258,0.002045,0.002228,0.001190,0.001831,0.001617,0.001434
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003021,0.002869,0.002869,0.002960,0.002747,0.003204,0.003052,0.003418,0.003418,0.003052


In [4]:
check.head()

,label,data1,data2,data3,data4,data5,data6,data7,data8,data9,...,data47095,data47096,data47097,data47098,data47099,data47100,data47101,data47102,data47103,data47104
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.000824,-0.000580,-0.001434,-0.001343,-0.001068,-0.001404,-0.001678,-0.001709,-0.001617,-0.001984
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001495,0.000946,0.000610,0.001373,0.000977,0.001648,0.001892,0.002136,0.002472,0.002380
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000671,0.000244,0.000305,0.000061,0.000031,0.000519,-0.000122,0.001038,0.000336,0.000244
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.000397,-0.001434,-0.000916,-0.001038,-0.000275,-0.000946,-0.001099,-0.000580,-0.000916,-0.000244
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000885,0.001892,0.001404,0.001007,0.001495,0.001312,0.000885,0.001068,0.000641,0.000763


In [5]:
#corner=1 wall=0
train_label=train['label']
check_label=check['label']
del train['label']
del check['label']

In [6]:
train_label.shape

(1379,)

In [7]:
check_label.shape

(420,)

In [8]:
train.shape

(1379, 47104)

In [9]:
check.shape

(420, 47104)

In [10]:
x_train=np.zeros((np.size(train,0),12,12))
tr=np.array(train)
ch=np.array(check)
for i in range(0,np.size(tr,0)):
    x_train[i,:,:]=librosa.feature.chroma_stft(tr[i,:], sr=44100, hop_length=4000)
x_check=np.zeros((np.size(check,0),12,12))
for i in range(0,np.size(ch,0)):
    x_check[i,:,:]=librosa.feature.chroma_stft(ch[i,:], sr=44100, hop_length=4000)

In [11]:
x_train.shape

(1379, 12, 12)

In [12]:
x_train = x_train.reshape(-1,12,12,1)
x_check = x_check.reshape(-1,12,12,1)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)

model = Sequential()
model.add(Conv2D(75, (3, 3), strides=1,padding="same", activation="relu", 
                 input_shape=(12,12,1)))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(units=10, activation="sigmoid"))
model.add(Dropout(0.1))
model.add(Dense(units=1, activation="sigmoid"))

In [21]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 12, 12, 75)        750       
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 75)        0         
                                                                 
 batch_normalization (BatchN  (None, 12, 12, 75)       300       
 ormalization)                                                   
                                                                 
 flatten (Flatten)           (None, 10800)             0         
                                                                 
 dense (Dense)               (None, 10)                108010    
                                                                 
 dropout_3 (Dropout)         (None, 10)                0         
                                                      

In [41]:
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'], optimizer='adamax')

In [42]:
model.fit(x_train,train_label,epochs=20)

Epoch 1/20
44/44 [==============================] - 1s 11ms/step - loss: 0.1320 - accuracy: 0.9637
Epoch 2/20
44/44 [==============================] - 1s 12ms/step - loss: 0.1018 - accuracy: 0.9761
Epoch 3/20
44/44 [==============================] - 1s 13ms/step - loss: 0.1049 - accuracy: 0.9739
Epoch 4/20
44/44 [==============================] - 0s 11ms/step - loss: 0.0864 - accuracy: 0.9862
Epoch 5/20
44/44 [==============================] - 0s 11ms/step - loss: 0.0909 - accuracy: 0.9804
Epoch 6/20
44/44 [==============================] - 1s 13ms/step - loss: 0.0829 - accuracy: 0.9833
Epoch 7/20
44/44 [==============================] - 1s 13ms/step - loss: 0.0906 - accuracy: 0.9840
Epoch 8/20
44/44 [==============================] - 1s 13ms/step - loss: 0.0852 - accuracy: 0.9840
Epoch 9/20
44/44 [==============================] - 1s 13ms/step - loss: 0.0812 - accuracy: 0.9884
Epoch 10/20
44/44 [==============================] - 1s 13ms/step - loss: 0.0761 - accuracy: 0.9884
Epoch 11/

In [52]:
model.evaluate(x_check,check_label)

14/14 [==============================] - 0s 6ms/step - loss: 0.7938 - accuracy: 0.6833


[0.7937926650047302, 0.6833333373069763]

(420, 12, 12, 1)